# Web scraping stat Viego

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [6]:
url = "https://leagueoflegends.fandom.com/wiki/Viego/LoL"
url

'https://leagueoflegends.fandom.com/wiki/Viego/LoL'

*Statistic growth = base+ (perlvl) x (n-1) x (0.7025 + 0.0175 x (n-1))*

In [7]:
response = requests.get(url)
html = BeautifulSoup(response.text, "html.parser")

In [8]:
def findSpan(typeTag : str,  libTag : str):
    return html.find("span", {typeTag : libTag})

In [9]:
hp = findSpan("id","Health_Viego")
hp_lvl = findSpan("id","Health_Viego_lvl")
hp_list = [(float(hp.text) + float(hp_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [10]:
armor =  findSpan("id","Armor_Viego")
armor_lvl = findSpan("id","Armor_Viego_lvl")
armor_list = [(float(armor.text) + float(armor_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [11]:
rm = findSpan("id","MagicResist_Viego")
rm_lvl = findSpan("id","MagicResist_Viego_lvl")
rm_list = [(float(rm.text) + float(rm_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [12]:
ad = findSpan("id","AttackDamage_Viego")
ad_lvl = findSpan("id","AttackDamage_Viego_lvl")
ad_list = [(float(ad.text) + float(ad_lvl.text) * (n-1) *(0.7025 +0.0175 * (n-1))) for n in range(1,19)]

In [13]:
attack_speed = html.findAll('div', {"style" : "width: calc(1 / 2 * 100%);"})
attack_speed = attack_speed[10]
attack_speed = float(re.sub(r'[^0-9.,]', '', attack_speed.text))
print(attack_speed)

0.658


In [14]:
bonus_as_lvl = html.findAll('div', {"style" : "width: calc(1 / 2 * 100%);"})
bonus_as_lvl = bonus_as_lvl[13]
bonus_as_lvl = float(re.sub(r'[^0-9.,]', '', bonus_as_lvl.text))
bonus_as_lvl /=100
print(bonus_as_lvl)
bonus_as_list = [ float(bonus_as_lvl) * (n-1) *(0.7025 +0.0175 * (n-1)) for n in range(1,19)]
print(bonus_as_list)

0.025
[0.0, 0.018, 0.036875000000000005, 0.05662500000000001, 0.07725, 0.09875, 0.12112500000000001, 0.14437500000000003, 0.1685, 0.19350000000000003, 0.21937500000000001, 0.24612500000000004, 0.27375000000000005, 0.30225, 0.33162500000000006, 0.36187500000000006, 0.393, 0.42500000000000004]


RATIO = 0.658
Formule total AS : *TOTAL = base + ratio*bonus

In [15]:
ratio = 0.658

In [16]:
as_list = [ attack_speed + ratio*bonus_as_list[n]  for n in range(0,18)]
print(as_list)

[0.658, 0.669844, 0.6822637500000001, 0.69525925, 0.7088305, 0.7229775, 0.7377002500000001, 0.75299875, 0.768873, 0.785323, 0.80234875, 0.8199502500000001, 0.8381275000000001, 0.8568805, 0.87620925, 0.89611375, 0.9165940000000001, 0.9376500000000001]


In [17]:
viegoStat = pd.DataFrame(index=range(1,19),columns=['health','armor','magic_res','ad','as'])
viegoStat['health'] = hp_list
viegoStat['armor'] = armor_list
viegoStat['magic_res'] = rm_list
viegoStat['ad'] = ad_list
viegoStat['as'] = bonus_as_list #Ici c'est le bonus d'attaque speed, il faut additionner les autres bonus
viegoStat

,health,armor,magic_res,ad,as
1,630.000,34.000,32.00000,57.0000,0.000000
2,708.480,37.744,33.47600,59.5200,0.018000
3,790.775,41.670,35.02375,62.1625,0.036875
4,876.885,45.778,36.64325,64.9275,0.056625
5,966.810,50.068,38.33450,67.8150,0.077250
6,1060.550,54.540,40.09750,70.8250,0.098750
7,1158.105,59.194,41.93225,73.9575,0.121125
8,1259.475,64.030,43.83875,77.2125,0.144375
9,1364.660,69.048,45.81700,80.5900,0.168500
10,1473.660,74.248,47.86700,84.0900,0.193500


Ultime Viego  

Ultime Cooldown 

In [67]:
soup = BeautifulSoup(response.content, 'html.parser')

ultime_cooldown = soup.find("div", {'class': 'skill skill_r'})
if ultime_cooldown:
    ultime_cooldown_value = ultime_cooldown.find('div', {'class': 'pi-data-value pi-font'})
    if ultime_cooldown_value:
        ultime_cooldown_text = ultime_cooldown_value.text
        print('COOLDOWN:', ultime_cooldown_text)
    else:
        print('Element "pi-data-value pi-font" non trouvé dans skill_r')
else:
    print('Element "skill skill_r" non trouvé dans la page')

COOLDOWN: 120 / 100 / 80


ligne rouge dans le paragraphe 

In [68]:
soup = BeautifulSoup(response.content, 'html.parser')
lr = soup.find("div", {'class' : 'skill skill_r'})
lr = lr.find('span', {'class' : 'pp-tooltip'})['data-displayformula']
print(lr)

120 x (1 + critical strike chance)


Physical Damage

In [69]:
soup = BeautifulSoup(response.content, 'html.parser')
pdamage = soup.find("div", {'class': 'skill skill_r'})
pdamage = pdamage.find('div', {'style': 'grid-column-end: span 2; display:contents'})
pdamage = pdamage.find('span', {'style': 'color: #1F995C; white-space:normal'})
print(pdamage.text)

12 / 16 / 20% (+ 5% per 100 bonus AD) of target's missing health


Stat board

In [79]:
UltimeStat = pd.DataFrame(index=range(1,4), columns=['Ultimate Cooldown', 'Physical Damage', 'Ligne Rouge'])
UltimeStat['Ultimate Cooldown'] = ultime_cooldown_text.split(" / ")
UltimeStat['Physical Damage'] = pdamage.text.split(" / ")
UltimeStat['Ligne Rouge'] = lr
UltimeStat

,Ultimate Cooldown,Physical Damage,Ligne Rouge
1,120,12,120 x (1 + critical strike chance)
2,100,16,120 x (1 + critical strike chance)
3,80,20% (+ 5% per 100 bonus AD) of target's missin...,120 x (1 + critical strike chance)
